# Operating LWRs
The whole goal of this notebook is to demonstrate the method used to pull the LWR deployment information from the EIA.

In [1]:
import numpy as np
import pandas as pd
import sys
import os
from collections import defaultdict
import textwrap
import xml.etree.ElementTree as ET

In [2]:
import scenario_definitions as sd

In [3]:
# %%
# the source of the LWR information for the csv is this eia table, only run if
# you don't have the csv
table = pd.read_html('https://www.eia.gov/nuclear/spent_fuel/ussnftab2.php')
long = table[0].to_dict()
long_df = pd.DataFrame(long)

#long_df.to_csv('lwr_info.csv')

In [4]:
long_df

,Reactor name,State,Reactor type,Reactor vendora,Core size (number of assemblies),Startup date (year) b,License expiration (year),Actual retirement (year)
0,Arkansas Nuclear One 1,AR,PWR,B&W,177,1974,2034,NaN
1,Arkansas Nuclear One 2,AR,PWR,CE,177,1978,2038,NaN
2,Beaver Valley 1,PA,PWR,WE,157,1976,2036,NaN
3,Beaver Valley 2,PA,PWR,WE,157,1987,2047,NaN
4,Big Rock Point,MI,BWR,GE,84,1964,NaN,1997
...,...,...,...,...,...,...,...,...
115,Wolf Creek 1,KS,PWR,WE,193,1985,2045,NaN
116,Yankee Rowe,MA,PWR,WE,76,1960,NaN,1991
117,Zion 1,IL,PWR,WE,193,1973,NaN,1997
118,Zion 2,IL,PWR,WE,193,1973,NaN,1996


In [5]:
def generate_facility_xml(df, reactor):
    """
    Generate the XML string for a reactor facility from the given dataframe.

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the information about the reactor.
    reactor : str
        The name of reactor to generate the XML for.

    Returns
    -------
    str
        The XML string for the reactor facility.

    Notes
    -----
    * This function assumes that LWRs will all operate for 80 years unless they are prematurely retired.
    * The user must confirm the latitude, longitude, and power capacity of each reactor.
    """
    # Find the index of the reactor in the dataframe
    reactor_index = df[df['Reactor name'] == reactor].index[0]

    # Extract the information about the reactor
    reactor_name = df['Reactor name'].iloc[reactor_index]
    startup_date = df['Startup date (year) b'].iloc[reactor_index]
    retirement_date = df['Actual retirement (year)'].iloc[reactor_index]
    core_size = df['Core size (number of assemblies)'].iloc[reactor_index]

    # If retirement_date is NaN, set it to 80 years after the startup date
    if pd.isna(retirement_date):
        retirement_date = int(startup_date) + 80
    else:
        retirement_date = int(retirement_date)

    # Calculate the lifetime of the reactor in months
    life_months = str((retirement_date - int(startup_date)) * 12)

    # Format the information into the desired XML structure
    xml_string = textwrap.dedent(f"""
<facility>
  <name>{reactor_name}</name>
  <lifetime>{life_months}</lifetime>
  <config>
    <Reactor>
      <fuel_incommods>  <val>fresh_uox</val> </fuel_incommods>
      <fuel_inrecipes>  <val>fresh_uox</val> </fuel_inrecipes>
      <fuel_outcommods> <val>used_uox</val> </fuel_outcommods>
      <fuel_outrecipes> <val>used_uox</val> </fuel_outrecipes>
      <cycle_time>18</cycle_time>
      <refuel_time>1</refuel_time>
      <assem_size>427.38589211618256</assem_size>
      <n_assem_core>{core_size}</n_assem_core>
      <n_assem_batch>80</n_assem_batch>
      <power_cap></power_cap>
      <longitude></longitude>
      <latitude></latitude>
    </Reactor>
  </config>
</facility>
""").strip()
    return xml_string

## An example of this function in action

In [6]:
generate_facility_xml(long_df, 'Arkansas Nuclear One 1')

'<facility>\n  <name>Arkansas Nuclear One 1</name>\n  <lifetime>960</lifetime>\n  <config>\n    <Reactor>\n      <fuel_incommods>  <val>fresh_uox</val> </fuel_incommods>\n      <fuel_inrecipes>  <val>fresh_uox</val> </fuel_inrecipes>\n      <fuel_outcommods> <val>used_uox</val> </fuel_outcommods>\n      <fuel_outrecipes> <val>used_uox</val> </fuel_outrecipes>\n      <cycle_time>18</cycle_time>\n      <refuel_time>1</refuel_time>\n      <assem_size>427.38589211618256</assem_size>\n      <n_assem_core>177</n_assem_core>\n      <n_assem_batch>80</n_assem_batch>\n      <power_cap></power_cap>\n      <longitude></longitude>\n      <latitude></latitude>\n    </Reactor>\n  </config>\n</facility>'

## Double check values with symbols
In the database, there are several reactors that have special characters next to their names. For our purposes, these symbols will be removed. I identified these through trial and error, I'm sure there was a more pythonic way to do that.

In [7]:
print(long_df.loc[33,'Reactor name'],',',long_df.loc[33,'Actual retirement (year)'])
long_df.loc[33,'Actual retirement (year)'] = '2020'

print(long_df.loc[48,'Reactor name'],',',long_df.loc[48,'Actual retirement (year)'])

long_df.loc[48,'Actual retirement (year)'] = '2020'

print(long_df.loc[78,'Reactor name'],',',long_df.loc[78,'Actual retirement (year)'])

long_df.loc[78,'Actual retirement (year)'] = '2019'

print(long_df.loc[105,'Reactor name'],',',long_df.loc[105,'Actual retirement (year)'])

long_df.loc[105,'Actual retirement (year)'] = '2019'

Duane Arnold , 2020*
Indian Point 2 , 2020*
Pilgrim 1 , 2019*
Three Mile Island 1 , 2019*


## Now we will generate all of the LWR files for the simulations

In [8]:
# specify the location these .xml files should be saved to relative to this one
output_dir = '../reactors/lwrs/'

# if the output directory does not exist, create it
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [9]:
# iterate through each reactor in the dataframe and generate an .xml file for it
# we exclude the last one, because that row is a source, not a reactor.
for reactor in long_df['Reactor name'].tolist()[0:-1]:
    xml_string = generate_facility_xml(long_df, reactor)
    with open(f'{output_dir}{reactor}.xml', 'w') as f:
        f.write(xml_string)

# Add the Reference Unit Power (MW)
The EIA database does not have the power capacity for each unit, so we can pull it from the PRIS Database (https://pris.iaea.org/PRIS/CountryStatistics/CountryDetails.aspx?current=US) and enter those values into the xml forms individually.

After these values have been added, we will add them to `lwr_info.csv`.

In [11]:
# This is the code I used to pull the power capacities from the xml files after
# I manually entered them. For your sake, I will just provide the dictionary
# and the code you can use to add it to the csv.


# Only run this is you have the reactors all ready set up with the power_cap
# variable. If you don't, you can just use the dictionary I provide below.

def extract_power_cap(directory):
    """
    Extract the power_cap values from the XML files in the given directory.

    Parameters
    ----------
    directory : str
        The directory containing the XML files.

    Returns
    -------
    dict
        A dictionary containing the power_cap values for each reactor.
    """

    # Initialize an empty dictionary to store the power_cap values
    power_cap_dict = {}

    # Iterate over each XML file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.xml'):
            file_path = os.path.join(directory, filename)

            # Parse the XML file
            tree = ET.parse(file_path)
            root = tree.getroot()

            # Extract the reactor name and power_cap value
            reactor_name = root.find('name').text
            power_cap_element = root.find('.//power_cap').text

            # Add the power_cap value to the dictionary
            power_cap_dict[reactor_name] = int(power_cap_element)

    return power_cap_dict

# Extract the power_cap values from the XML files
power_cap_dict = extract_power_cap(output_dir)

In [12]:
# This dictionary contains the power capacities for each LWR.
power_cap_dict = {'Arkansas Nuclear One 1': 836,
 'Arkansas Nuclear One 2': 988,
 'Beaver Valley 1': 908,
 'Beaver Valley 2': 905,
 'Big Rock Point': 67,
 'Braidwood 1': 1194,
 'Braidwood 2': 1160,
 'Browns Ferry 1': 1200,
 'Browns Ferry 2': 1200,
 'Browns Ferry 3': 1210,
 'Brunswick 1': 938,
 'Brunswick 2': 932,
 'Byron 1': 1164,
 'Byron 2': 1136,
 'Callaway': 1215,
 'Calvert Cliffs 1': 877,
 'Calvert Cliffs 2': 855,
 'Catawba 1': 1160,
 'Catawba 2': 1150,
 'Clinton 1': 1062,
 'Columbia': 1131,
 'Comanche Peak 1': 1205,
 'Comanche Peak 2': 1195,
 'Cook 1': 1030,
 'Cook 2': 1168,
 'Cooper Station': 769,
 'Crystal River 3': 860,
 'Davis-Besse': 894,
 'Diablo Canyon 1': 1138,
 'Diablo Canyon 2': 1118,
 'Dresden 1': 197,
 'Dresden 2': 894,
 'Dresden 3': 879,
 'Duane Arnold': 601,
 'Enrico Fermi 2': 1115,
 'Farley 1': 874,
 'Farley 2': 883,
 'Fitzpatrick': 813,
 'Fort Calhoun': 482,
 'Ginna': 560,
 'Grand Gulf 1': 1401,
 'Haddam Neck': 560,
 'Harris 1': 964,
 'Hatch 1': 876,
 'Hatch 2': 883,
 'Hope Creek': 1172,
 'Humboldt Bay': 63,
 'Indian Point 1': 257,
 'Indian Point 2': 998,
 'Indian Point 3': 1030,
 'Kewaunee': 566,
 'La Crosse': 48,
 'LaSalle County 1': 1137,
 'LaSalle County 2': 1140,
 'Limerick 1': 1134,
 'Limerick 2': 1134,
 'Maine Yankee': 860,
 'McGuire 1': 1158,
 'McGuire 2': 1158,
 'Millstone 1': 641,
 'Millstone 2': 869,
 'Millstone 3': 1210,
 'Monticello': 628,
 'Nine Mile Point 1': 613,
 'Nine Mile Point 2': 1277,
 'North Anna 1': 948,
 'North Anna 2': 944,
 'Oconee 1': 847,
 'Oconee 2': 848,
 'Oconee 3': 859,
 'Oyster Creek': 619,
 'Palisades': 805,
 'Palo Verde 1': 1311,
 'Palo Verde 2': 1314,
 'Palo Verde 3': 1312,
 'Peach Bottom 2': 1300,
 'Peach Bottom 3': 1331,
 'Perry 1': 1240,
 'Pilgrim 1': 677,
 'Point Beach 1': 591,
 'Point Beach 2': 591,
 'Prairie Island 1': 522,
 'Prairie Island 2': 519,
 'Quad Cities 1': 908,
 'Quad Cities 2': 911,
 'Rancho Seco': 873,
 'River Bend 1': 967,
 'Robinson 2': 741,
 'SHOREHAM': 820,
 'Salem 1': 1169,
 'Salem 2': 1158,
 'San Onofre 1': 436,
 'San Onofre 2': 1070,
 'San Onofre 3': 1080,
 'Seabrook': 1246,
 'Sequoyah 1': 1152,
 'Sequoyah 2': 1139,
 'South Texas 1': 1280,
 'South Texas 2': 1280,
 'St. Lucie 1': 981,
 'St. Lucie 2': 987,
 'Summer 1': 973,
 'Surry 1': 838,
 'Surry 2': 838,
 'Susquehanna 1': 1257,
 'Susquehanna 2': 1257,
 'THREE_MILE_ISLAND-2': 880,
 'Three Mile Island 1': 819,
 'Trojan': 1095,
 'Turkey Point 3': 837,
 'Turkey Point 4': 821,
 'Vermont Yankee': 605,
 'Vogtle 1': 1150,
 'Vogtle 2': 1117,
 'Waterford 3': 1168,
 'Watts Bar 1': 1157,
 'Watts Bar 2': 1164,
 'Wolf Creek 1': 1200,
 'Yankee Rowe': 167,
 'Zion 1': 1040,
 'Zion 2': 1040}

To add power_cap_dict to `lwr_info.csv`, run the following blocks.

In [13]:
def add_power_cap(df, power_cap_dict):
    """
    Create a new column 'power_cap' in long_df and populate it with values from power_cap_dict

    Parameters
    ----------
    df : pandas.DataFrame
        The dataframe containing the information about the reactors.
    power_cap_dict : dict
        A dictionary mapping reactor names to power capacities.

    Returns
    -------
    df : pandas.DataFrame
        The input dataframe with the 'power_cap' column added
    """
    df['power_cap'] = df['Reactor name'].map(power_cap_dict)

    return df

In [14]:
add_power_cap(long_df, power_cap_dict)
long_df.to_csv('lwr_info.csv', index=False)

,Reactor name,State,Reactor type,Reactor vendora,Core size (number of assemblies),Startup date (year) b,License expiration (year),Actual retirement (year),power_cap
0,Arkansas Nuclear One 1,AR,PWR,B&W,177,1974,2034,NaN,836.0
1,Arkansas Nuclear One 2,AR,PWR,CE,177,1978,2038,NaN,988.0
2,Beaver Valley 1,PA,PWR,WE,157,1976,2036,NaN,908.0
3,Beaver Valley 2,PA,PWR,WE,157,1987,2047,NaN,905.0
4,Big Rock Point,MI,BWR,GE,84,1964,NaN,1997,67.0
...,...,...,...,...,...,...,...,...,...
115,Wolf Creek 1,KS,PWR,WE,193,1985,2045,NaN,1200.0
116,Yankee Rowe,MA,PWR,WE,76,1960,NaN,1991,167.0
117,Zion 1,IL,PWR,WE,193,1973,NaN,1997,1040.0
118,Zion 2,IL,PWR,WE,193,1973,NaN,1996,1040.0


# Add new Vogtle reactors
Presently, units 3 and 4 at Vogtle are not included in the EIA database. Run the following code to add them to this repository.

In [15]:
vogtle_3 = textwrap.dedent(f"""
<facility>
  <name>Vogtle 3</name>
  <lifetime>960</lifetime>
  <config>
    <Reactor>
      <fuel_incommods>  <val>fresh_uox</val> </fuel_incommods>
      <fuel_inrecipes>  <val>fresh_uox</val> </fuel_inrecipes>
      <fuel_outcommods> <val>used_uox</val> </fuel_outcommods>
      <fuel_outrecipes> <val>used_uox</val> </fuel_outrecipes>
      <cycle_time>18</cycle_time>
      <refuel_time>1</refuel_time>
      <assem_size>427.38589211618256</assem_size>
      <n_assem_core>193</n_assem_core>
      <n_assem_batch>80</n_assem_batch>
      <power_cap>1117</power_cap>
      <longitude>-81.7606</longitude>
      <latitude>33.1433</latitude>
    </Reactor>
  </config>
</facility>
""")

# Use os.path.join for proper path concatenation
file_path = os.path.join(output_dir, "Vogtle 3.xml")

with open(file_path, 'w') as f:
    f.write(vogtle_3)

In [16]:
vogtle_4 = textwrap.dedent(f"""
<facility>
  <name>Vogtle 4</name>
  <lifetime>960</lifetime>
  <config>
    <Reactor>
      <fuel_incommods>  <val>fresh_uox</val> </fuel_incommods>
      <fuel_inrecipes>  <val>fresh_uox</val> </fuel_inrecipes>
      <fuel_outcommods> <val>used_uox</val> </fuel_outcommods>
      <fuel_outrecipes> <val>used_uox</val> </fuel_outrecipes>
      <cycle_time>18</cycle_time>
      <refuel_time>1</refuel_time>
      <assem_size>427.38589211618256</assem_size>
      <n_assem_core>193</n_assem_core>
      <n_assem_batch>80</n_assem_batch>
      <power_cap>1117</power_cap>
      <longitude>-81.7606</longitude>
      <latitude>33.1433</latitude>
    </Reactor>
  </config>
</facility>
""")

# Use os.path.join for proper path concatenation
file_path = os.path.join(output_dir, "Vogtle 4.xml")

with open(file_path, 'w') as f:
    f.write(vogtle_4)